In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import corner
import matplotlib.pyplot as plt

import hemcee
from hemcee.sampler import TFModel
from maelstrom.kepler import kepler

/Users/danielhey/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
#kicid=8311110 # Weird? PB1/SB2
kicid=9651065 # PB1/SB1


#kicid = 9837267

#kicid = 8311110  # SB1
#kicid = 6862920  # PB2/SB2 high e
#kicid = 10080943  #PB2/SB2 low e
#kicid = 4471379 # SB0

In [11]:
times, dmag = np.loadtxt(f"data/kic{kicid}_lc.txt",usecols=(0,1)).T
time_mid = (times[0] + times[-1]) / 2.
times -= time_mid
dmmags = dmag * 1000. 

metadata = np.loadtxt(f"data/kic{kicid}_metadata.csv", delimiter=",", skiprows=1)
nu_arr = metadata[::6]

orbits = pd.read_csv(f"data/orbits.csv").rename(columns = lambda x: x.strip())

orb_params = orbits[orbits.Name == f"kic{kicid}"].iloc[0]
porb = orb_params.Porb
a1 = orb_params["a1sini/c"] 
tp = orb_params["t_p"] - time_mid 
e = orb_params["e"]
varpi = orb_params["varpi"]
a1d = a1#/86400.0

In [12]:
class BoundParam(object):
    def __init__(self, name, value, min_value, max_value, dtype=tf.float64):
        self.name = name
        self.value = value
        self.min_value = min_value
        self.max_value = max_value
        
        # Bound
        self.param = tf.Variable(self.get_bounded_for_value(self.value, self.min_value, self.max_value), dtype=dtype, name=name + "_param")
        self.var = self.min_value + (self.max_value - self.min_value) / (1.0 + tf.exp(-self.param))
        
        self.log_jacobian = tf.log(self.var - self.min_value) + tf.log(self.max_value - self.var) - np.log(self.max_value - self.min_value)
        # Add this to the log prior
        self.log_prior = tf.reduce_sum(self.log_jacobian)
    def get_bounded_for_value(self, value, min_val, max_val):
        # Check if correct bounds
        if np.any(value <= min_val) or np.any(value >= max_val):
            raise ValueError("Value must be within the given bounds")
        return np.log(value-min_val)-np.log(max_val-value)
    
    def get_value_for_bounded(self,param):
        return self.min_value + (self.max_value - self.min_value) / (1.0 + np.exp(-param))

In [13]:
sess = tf.InteractiveSession()
T = tf.float64

# Unbound tensors
nu_tensor = tf.Variable(nu_arr, dtype=T)

# Bound tensors
porb_tensor = BoundParam('Porb', porb, 1, 500)  # Orbital period
varpi_tensor = BoundParam('Varpi', varpi, 0, 2*np.pi)   # Angle of the ascending node
tp_tensor = BoundParam('t_p', tp, -1000, 0) # Time of periastron
e_tensor = BoundParam('e', e, 1e-10, 0.99) # Eccentricity
log_sigma2_tensor = BoundParam('log_sigma2', 0., -5, 5) # Known value
a1d_tensor = BoundParam('a_1d', a1d, 0., 300.) # Projected semimajor axis

if rv:
    # Tensors specific to SB1/2
    # Some notes on gammav: 
    # If gammav is specified as gammav/c then scipy fit will work fine
    gammav_tensor = BoundParam('gammav',np.mean(rv_RV),-100,100)
    log_rv_sigma2_tensor = BoundParam('logrv', 0., -0.1,0.1)

/Users/danielhey/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [14]:
times_tensor = tf.placeholder(T, times.shape)
dmmags_tensor = tf.placeholder(T, dmmags.shape)

    
# Solve Kepler's equation
mean_anom = 2.0 * np.pi * (times_tensor - tp_tensor.var) / porb_tensor.var
ecc_anom = kepler(mean_anom, e_tensor.var)
true_anom = 2.0 * tf.atan2(tf.sqrt(1.0+e_tensor.var)*tf.tan(0.5*ecc_anom),tf.sqrt(1.0-e_tensor.var) + tf.zeros_like(times_tensor))

# Here we define how the time delay will be calculated:
tau_tensor = -(a1d_tensor.var / 86400) * (1.0 - tf.square(e_tensor.var)) * tf.sin(true_anom + varpi_tensor.var) / (1.0 + e_tensor.var*tf.cos(true_anom))

# And the design matrix:
arg_tensor = 2.0 * np.pi * nu_tensor[None, :] * (times_tensor - tau_tensor)[:, None]
D_tensor = tf.concat([tf.cos(arg_tensor), tf.sin(arg_tensor)], axis=1)

# Define the linear solve for W_hat:
DTD_tensor = tf.matmul(D_tensor, D_tensor, transpose_a=True)
DTy_tensor = tf.matmul(D_tensor, dmmags_tensor[:, None], transpose_a=True)
W_hat_tensor = tf.linalg.solve(DTD_tensor, DTy_tensor)

# Finally, the model and the chi^2 objective:
model_tensor = tf.squeeze(tf.matmul(D_tensor, W_hat_tensor)) # Removes dimensions of size 1 from the shape of a tensor.
# Sometimes faster with switched signs on log_sigma2 here:
chi2_tensor = tf.reduce_sum(tf.square(dmmags_tensor - model_tensor)) * tf.exp(-log_sigma2_tensor.var)
chi2_tensor += len(times) * (log_sigma2_tensor.var)


init = tf.global_variables_initializer()
sess.run(init)

In [15]:
feed_dict = {
    times_tensor: times,
    dmmags_tensor: dmmags
}
# Not working simultaneously:
# varpi & tp

var = [
    porb_tensor, # PORB OK
    varpi_tensor, # BY ITSELF: OK, 1.05
    tp_tensor, # BY ITSELF: OK, -320.20
    a1d_tensor,
    e_tensor, # NOT OK
    log_sigma2_tensor,
]
    
var_list = [tensors.param for tensors in var]

In [16]:
for i in var:
    print(i.name, ":", i.value, ':', i.get_value_for_bounded(sess.run(i.param)))

opt = tf.contrib.opt.ScipyOptimizerInterface(chi2_tensor, var_list=var_list)
for i in range(10):
    opt.minimize(sess, feed_dict=feed_dict)

for i,tensor in enumerate(var):
    tensor.real = tensor.get_value_for_bounded(sess.run(tensor.param))

for i in var:
    print(i.name, ":", np.round(i.value,5), ':', i.get_value_for_bounded(sess.run(i.param)))
if rv:
    rv_phi_test = np.sort(np.linspace(0, porb_tensor.real, 5000) % porb_tensor.real)
    vrad_test = sess.run(vrad_tensor, feed_dict={rv_time_tensor: rv_phi_test})
    plt.errorbar((rv_JD % porb_tensor.real)/porb_tensor.real,rv_RV,rv_err,fmt=".",label='RV obs')
    plt.plot(rv_phi_test/porb_tensor.real, vrad_test,label='RV th')
    plt.xlabel("Orbital phase")
    plt.ylabel("RV (km/s)")
    plt.legend()
    plt.show()

Porb : 272.425128 : 272.425128
Varpi : 2.198662 : 2.1986620000000006
t_p : -543.9320410500077 : -543.9320410500077
a_1d : 184.55653700000002 : 184.55653700000002
e : 0.466711 : 0.466711
log_sigma2 : 0.0 : 0.0
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 87109.843877
  Number of iterations: 30
  Number of functions evaluations: 33
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 87109.843876
  Number of iterations: 1
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 87109.843876
  Number of iterations: 1
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 87109.843876
  Nu

In [17]:

# Calculate prior
log_prior = tf.constant(0.0, dtype=tf.float64)
# Add the jacobian to the prior
for tensor in var:
    if tensor.log_prior is not None:
        log_prior += tensor.log_prior

log_prob = - 0.5 * chi2_tensor + log_prior

model = TFModel(log_prob, var_list=var_list, feed_dict=feed_dict)
model.setup()
coords = model.current_vector()

In [ ]:
metric = hemcee.metric.DenseMetric(np.eye(len(coords)))
step = hemcee.step_size.VariableStepSize()
sampler = hemcee.NoUTurnSampler(model.value, model.gradient, step_size=step, metric=metric)

# Burn-in
results = sampler.run_warmup(coords, 2500, tune_metric=True)

# Run the sampler
coords_chain, logprob_chain = sampler.run_mcmc(
        results[0], 
        5000, 
        initial_log_prob=results[1], 
        var_names='', 
        plot=False, update_interval=100, 
        tune=False
        )

initial warm up: step_size: 3.9e-03; mean(accept_stat): 0.455:  77%|███████▋  | 77/100 [02:34<00:46,  2.00s/it]

In [ ]:
#taus = np.array([hemcee.autocorr.integrated_time(coords_chain[:, i])[0] for i in range(len(coords))])
#print("Mean autocorrelation time: {0}".format(np.mean(taus)))
plt.plot([coord[0] for coord in coords_chain])
plt.title('$P_{orb}$ trace')

# import pandas as pd
# df = pd.DataFrame(data=logprob_chain)
# df.to_csv('logprob_chain.csv')

for i,tensor in enumerate(var):
    tensor.real = tensor.get_value_for_bounded(coords_chain[:,i])

ndim = len(coords)
var_real = [tensor.real for tensor in var]
figure = corner.corner(list(zip(*var_real)),
                       labels=[tensor.name for tensor in var],
                       #quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 12})


true_vars = [porb, varpi, tp, e, 0, a1d]
true_vars = [tensor.value for tensor in var]
sample_vars = [np.median(tensor.real) for tensor in var]

axes = np.array(figure.axes).reshape((ndim, ndim))
for i in range(len(var_list)):   
    ax = axes[i, i]
    ax.axvline(true_vars[i], color="b")
    ax.axvline(sample_vars[i], color="r")

for yi in range(len(var_list)):
    for xi in range(yi):
        ax = axes[yi, xi]
        ax.axvline(sample_vars[xi], color="r")
        ax.axvline(true_vars[xi], color="b")
        ax.axhline(sample_vars[yi], color="r")
        ax.axhline(true_vars[yi], color="b")

if rv:
    fig = plt.figure()
    rv_phi_test = np.sort(np.linspace(0, np.mean(porb_tensor.real), 5000) % np.mean(porb_tensor.real))
    vrad_test = sess.run(vrad_tensor, feed_dict={rv_time_tensor: rv_phi_test})
    plt.errorbar((rv_JD % np.mean(porb_tensor.real))/np.mean(porb_tensor.real),rv_RV,rv_err,fmt=".",label='RV obs')
    plt.plot(rv_phi_test/np.mean(porb_tensor.real), vrad_test,label='RV th')
    plt.xlabel("Orbital phase")
    plt.ylabel("RV (km/s)")
    plt.legend()
    plt.show()